# Importing Relevant Libraries

In [1]:
#Mathematical Libraries
import numpy as np
import mpmath as mp 
import math
from decimal import Decimal as D

#Scipy
import scipy.special as sc
from scipy.integrate import quad
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline

#Plotting Library
import matplotlib.pyplot as plt
import matplotlib

#Miscellaneous Libraries
import time
from IPython.display import clear_output
import csv
import copy


# Class Definitions

In [2]:
#~~~~~~~~~~~~~~~~~~~~~Class definition: PopIII stars~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


class PopIIIStar:
    '''Describes important parameters of a population III star,
    Units:
            M - Solar
            R - Solar
            L - Solar
            Tc - Kelvin (K)
            rhoc - g/cm^3
            life_star - years'''
    def __init__(self, M = 0, R = 0, L = 0, Tc = 0, rhoc = 0, life_star = 0):
        self.mass = M
        self.radius = R
        self.lum = L
        self.core_temp = Tc
        self.core_density = rhoc
        self.lifetime = life_star
        
    #Calculates stellar volume
    def get_vol(self):
        vol = (4/3) * np.pi * (self.radius*6.96e10)**3 #in cm^3
        
        return vol
    
    def get_num_density(self):
        mn_grams = 1.6726e-24
        M = 1.9885e33 * self.mass
        
        n_baryon = 0.75*M/mn_grams * 1/(self.get_vol())
        
        return n_baryon
    
    def get_mass_grams(self):
        M_gram = 1.9885e33 * self.mass
        return M_gram
    
    def get_radius_cm(self):
        R_cm = self.radius*6.96e10
        return R_cm
    
    def get_vesc_surf(self):
        G  = 6.6743*10**(-8) #cgs units
        M = self.get_mass_grams()
        R = self.get_radius_cm()
        Vesc = np.sqrt(2*G*M/R) # escape velocity(cm/s) 
        return Vesc

# 100, 300, and 1000 Solar Mass Pop III Parameters

In [3]:
#Stellar params
M100 = PopIIIStar(100, 10**0.6147, 10**6.1470, 1.176e8, 32.3, 10**6)
M300 = PopIIIStar(300, 10**0.8697, 10**6.8172, 1.245e8, 18.8, 10**6)
M1000 = PopIIIStar(1000, 10**1.1090, 10**7.3047, 1.307e8, 10.49, 10**6)
stars_list = (M100, M300, M1000)

### Polytrope Functions - (Tau, Central Density, Potential, Density Profile, Effective Volumes)

In [4]:
######################### CORRECT FUNCTIONS FOR POLYTROPES ###############################################################

#Retrieves solution to laneEmden n=3 
def retrieve_LaneEmden():
    xis = []
    theta_arr = []
    with open('Lane_Emden.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            xis.append(float(row[0]))
            theta_arr.append(float(row[1]))
            
    return (xis, theta_arr)


#Retrieves tau(mx) from stored data
def retrieve_tau(star):
    mx = []
    tau = []
    with open('tau_mx_M%i.csv'%star.mass) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            mx.append(float(row[0]))
            tau.append(float(row[1]))
            
    return (mx, tau)

tau_fit_funcs = []
#Tau fits and function that will give output based on mx and star
for star in stars_list:
    mx_tau_fit, tau_temp = retrieve_tau(star)
    tau_fit_funcs.append(UnivariateSpline(mx_tau_fit, tau_temp, k = 5, s = 0))

def tau_fit(mx, star): #Returns tau from fitting function based on star and dm mass
    if(mx > 100):
        tau_val = 1
    else:
        if(star.mass == 100):
            tau_val = tau_fit_funcs[0](mx)
        elif(star.mass == 300):
            tau_val = tau_fit_funcs[1](mx)
        elif(star.mass == 1000):
            tau_val = tau_fit_funcs[2](mx)
        else:
            tau_val = 1
    return tau_val
    


# Solution to laneEmden Equation
xis, theta_arr = retrieve_LaneEmden()
# interpolating points for theta function
theta = UnivariateSpline(xis, theta_arr, k = 5, s = 0)


#Density at center of polytrope
def polytrope3_rhoc(star):
    
    #Getting stellar params
    Mstar = star.get_mass_grams() #grams
    Rstar = star.get_radius_cm()  #cm
    
    #x-intercept of the theta function
    xi_1 = xis[-1]

    #Slope of laneEmden at Theta = 0
    deriv_xi1 = theta.derivatives(xis[-1])[1]  
    
    #Central polytropic density as per n=3 polytropic model
    rhoc_poly = (-1/(4*np.pi)) * ((xi_1/Rstar)**3) * (Mstar/(xi_1**2)) * (deriv_xi1)**-1 #g/cm^3
    
    return rhoc_poly





#Polytropic potential
def potential_poly(xi, star):
    G = 6.6743*10**(-8) # gravitational constant in cgs units
    
    phi_xi = 4*np.pi*G*(polytrope3_rhoc(star))*(star.get_radius_cm()/xis[-1])**2 * (1 - theta(xi)) #cgs units
    
    return phi_xi




#Isotropic DM distribution using potential from n=3 polytrope
def nx_xi(mx, xi, star): #Normalized
    
    kb = 1.380649e-16 #Boltzmann constant in cgs Units (erg/K)
    
    #Finding Tx using Temperature function
    Tx = tau_fit(mx, star) * 10**8 #K

    #mx in g
    mx_g = mx*1.783e-24
    
    #Numerical DM number density profile for each DM mass (normalized)
    nx_xi_val = np.exp(-mx_g*potential_poly(xi, star)/(kb*Tx)) 
    
    return nx_xi_val





#Defining integrand for effective volumes
def integrand_Vj_poly3(xi, j, mx, star):
    return xi**2 * (nx_xi(mx, xi, star))**j
#Numerically integrating to get effective volumes for polytropes
def Vj_poly3(j, mx, star):
    xi_1 = xis[-1]
    factor = 4*np.pi*(star.get_radius_cm()/xi_1)**3 #Outside integral factor
    int_val = quad(integrand_Vj_poly3, 0, xi_1, args=(j, mx, star)) #Integrate nx/nc * xi**2 over star
    Vj = factor * int_val[0] #cm^3
    return Vj




# K factor for correcting bounds on sigv (due to breakdown in the assumption of n_c = nc_bar)
# NOT CURRENTLY USING, I AM DOING FULL NUMERICAL INTEGRATION INSTEAD
def k_factor(mx):
    
    #Dimensionless DM mass
    mu = mx/0.93827
    #Dimensionless DM temperature
    tau = tau_fit(mx, star)
    
    #Finding the first 0 of lane-emden 'xi_1'
    xi_1 = xis[-1]
    
    #Calculate "B" function that appears in exponents for the calculation of K constant
    B = 8*mu/tau * (1 - np.array(theta_arr))
    
    #Calculating integrands and using interpolating fnctions to fit them for ease of integration
    integrand_top = np.multiply(np.multiply(np.array(xis)**2, np.exp(-B)) ,np.array(theta_arr)**3 )
    integrand_bottom = np.multiply(np.array(xis)**2, np.exp(-B))
    integrandTop_spl = UnivariateSpline(xis, integrand_top, k = 5, s = 0)
    integrandBottom_spl = UnivariateSpline(xis, integrand_bottom, k = 5, s = 0)
    
    #Calculating K
    K = xi_1**3/6 * integrandTop_spl.integral(0, xi_1)/integrandBottom_spl.integral(0, xi_1)
    
    return K




### Evaporation Functions

In [5]:

#FITTING FUNCTION FOR theta**3
theta_cube = UnivariateSpline(xis, np.array(theta_arr)**3, k = 5, s = 0)


#Escape velocity of n=3 polytrope at given radius (dimensionless xi)
def vesc_r_poly(xi, star):
    G = 6.6743*10**(-8) # gravitational constant in cgs units
    
    xi1 =  xis[-1]

    vesc_xi = np.sqrt( 2*G*star.get_mass_grams()/star.get_radius_cm() + 2*(potential_poly(xi1, star) - potential_poly(xi, star)) )
    
    return vesc_xi/star.get_vesc_surf()
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




## Mu Functions ##
def mu(mx):#take mx in GeV
    mu_val = mx/0.93827
    return mu_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~





def mu_plus_minus(plus_minus, mx):
    if(plus_minus == '+'):
        mu_plus_val = (mu(mx) + 1)/2
    elif(plus_minus == '-'):
        mu_plus_val = (mu(mx) - 1)/2
    return mu_plus_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~






#l(r), most probable dimensionless velocity of protons at specific point in star
def proton_speed(xi, star):
    kb = 1.380649e-16 #Boltzmann constant in cgs Units (erg/K)
    Tc = 10**8 #Central star temperature taken to be ~ 10^8 K

    u = np.sqrt(2*kb*Tc*theta(xi)/1.6726219e-24) #cm/s (cgs units)
    
    l = u/star.get_vesc_surf()
    
    return l
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~






## Alpha/Beta Functions ##
def alpha(plus_minus, mx, q, z, xi, star):
    l = proton_speed(xi, star)
    if (plus_minus == '+'):
        alpha_val = (mu_plus_minus('+', mx)*q + mu_plus_minus('-', mx)*z)/l
    elif(plus_minus == '-'):
        alpha_val = (mu_plus_minus('+', mx)*q - mu_plus_minus('-', mx)*z)/l
        
    return alpha_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


def beta(plus_minus, mx, q, z, xi, star):
    l = proton_speed(xi, star)
    if (plus_minus == '+'):
        beta_val = (mu_plus_minus('-', mx)*q + mu_plus_minus('+', mx)*z)/l
    elif(plus_minus == '-'):
        beta_val = (mu_plus_minus('-', mx)*q - mu_plus_minus('+', mx)*z)/l
        
    return beta_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~





## Chi Function ##
def chi_func(a,b):
    chi_val = np.sqrt(np.pi)/2 * (mp.erf(b) - mp.erf(a))
    return chi_val

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 
    
    
    
#Number density of proton distribution in n = 3 polytrope
def eta_proton(xi):    
    eta_xi = theta_cube(xi)
    
    return eta_xi

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~





#R_+ coefficient
def R_plus(q, z, mx, xi, sigma, star):
    
    #Dimensionless Quantities
    mu_val = mu(mx)
    mu_plus = mu_plus_minus('+', mx)
    eta = eta_proton(xi)
    alpha_minus = alpha('-', mx, q, z, xi, star)
    alpha_plus = alpha('+', mx, q, z, xi, star)
    beta_minus = beta('-', mx, q, z, xi, star)
    beta_plus = beta('+', mx, q, z, xi, star)
    chi_alpha = chi_func(alpha_minus, alpha_plus)
    chi_beta = chi_func(beta_minus, beta_plus)
    l = proton_speed(xi, star)
    
    #Central proton number density (cm^-3)
    nc = polytrope3_rhoc(star)*0.75/1.6726e-24
    
    #R_plus calculation
    R_plus_val = 2*nc/np.sqrt(np.pi) * mu_plus**2/mu_val * q/z * eta * sigma * (chi_alpha + chi_beta*mp.exp(mu_val * (z**2 - q**2)/(l**2))) # cm^-1
    
    return R_plus_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



#R_- coefficient
def R_minus(q, z, mx, xi, sigma, star):
    
    #Dimensionless Quantities
    mu_val = mu(mx)
    mu_plus = mu_plus_minus('+', mx)
    eta = eta_proton(xi)
    alpha_minus = alpha('-', mx, q, z, xi, star)
    alpha_plus = alpha('+', mx, q, z, xi, star)
    beta_minus = beta('-', mx, q, z, xi, star)
    beta_plus = beta('+', mx, q, z, xi, star)
    chi_alpha = chi_func(-1*alpha_minus, alpha_plus)
    chi_beta = chi_func(-1*beta_minus, beta_plus)
    l = proton_speed(xi, star)
    
    #Central proton number density (cm^-3)
    nc = polytrope3_rhoc(star)*0.75/1.6726e-24
    
    #R_plus calculation
    R_minus_val = 2*nc/np.sqrt(np.pi) * mu_plus**2/mu_val * q/z * eta * sigma * (chi_alpha + chi_beta*mp.exp(mu_val * (z**2 - q**2)/(l**2))) # cm^-1
        
    return R_minus_val

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




#Omega_plus function, integral over R_plus
def omega_plus(z, mx, xi, sigma, star):
    vesc = star.get_vesc_surf()
    
    omega_plus_val = vesc * mp.quad(lambda q: R_plus(q, z, mx, xi, sigma, star), [vesc_r_poly(xi, star), np.inf]) #s^-1
    
    return omega_plus_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



#Omega function, integral over R_plus + Rminus
def omega(z, mx, xi, sigma, star):
    vesc = star.get_vesc_surf()
    
    
    omega_val = vesc * ( mp.quad(lambda q: R_minus(q, z, mx, xi, sigma, star), [vesc_r_poly(xi, star), z]) + mp.quad(lambda q: R_plus(q, z, mx, xi, sigma, star), [z, np.inf]) ) #s^-1
    
    return omega_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~






#Average dm speed in star (isotropic)
def dm_speed(mx, star):
    kb = 1.380649e-16 #Boltzmann constant in cgs Units (erg/K)
    Tx = tau_fit(mx, star) * 10**8 #DM temperature
    mx_g = mx * 1.783e-24 #Converting GeV/c^2 to g

    
    vx = np.sqrt(2*kb*Tx/mx_g) #cm/s
    
    ux = vx/star.get_vesc_surf() #Dimensionless
    
    return ux
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




def f_x(z, mx, xi, star):
    
    #Dm dimensionless speed
    ux = dm_speed(mx, star)
    ue_xi = vesc_r_poly(xi, star)
    vesc = star.get_vesc_surf()
    
    
    f_x_val = np.exp(-z**2/ux**2) * ( np.pi**(3/2) * ux**3 * vesc**3 * (sc.erf(ue_xi/ux) - 2/np.sqrt(np.pi)*ue_xi/ux*np.exp(-ue_xi**2/ux**2) )  )**-1

    
    return f_x_val
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def f_x_inf(z, mx, xi, star):
    
    #Dm dimensionless speed
    ux = dm_speed(mx, star)
    vesc = star.get_vesc_surf()
    
    
    f_x_val = mp.exp(-z**2/ux**2) * ( np.pi**(3/2) * ux**3 * vesc**3 )**-1

    
    return f_x_val



def integrand(z, mx, xi, sigma, star, vcut_inf = False):
    return f_x(z, mx, xi, star)*omega_plus(z, mx, xi, sigma, star)*z**2
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def integrand_inf(z, mx, xi, sigma, star, vcut_inf = False):
    return f_x_inf(z, mx, xi, star)*omega(z, mx, xi, sigma, star)*z**2




def R_integrated(mx, xi, sigma, star, vcut_inf = False):
    vesc = star.get_vesc_surf()
    if(vcut_inf == False):
        R = 4*np.pi * vesc**3 * quad(integrand, 0, vesc_r_poly(xi, star), args=(mx, xi, sigma, star, vcut_inf))[0]
    else:
        R = 4*np.pi * vesc**3 * mp.quad(lambda z: integrand_inf(z, mx, xi, sigma, star, vcut_inf), [0, np.inf])
    
    return R
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




def upper_integrand(xi, mx, sigma, star, vcut_inf = False):
    return xi**2 * nx_xi(mx, xi, star) * R_integrated(mx, xi, sigma, star, vcut_inf)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def lower_integrand(xi, mx, sigma, star):
    return xi**2 * nx_xi(mx, xi, star)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


def evap_coeff(mx, sigma, star, vcut_inf = False):
    xi1 = xis[-1]
    E = quad(upper_integrand, 0, xi1, args=(mx, sigma, star, vcut_inf))[0]/quad(lower_integrand, 0, xi1, args=(mx, sigma, star))[0]
    
    return E
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



#### Gould Evap Functions

In [6]:
import matplotlib.colors as colors

# set up some ploting stuff
fig = plt.figure(figsize = (12,8))
plt.style.use('fast')
palette = plt.get_cmap('magma')
palette1 = plt.get_cmap('viridis')


# polytrope definitions
M100 = PopIIIStar(100, 10**0.6147, 10**6.1470, 1.176e8, 32.3, 10**6)
M300 = PopIIIStar(300, 10**0.8697, 10**6.8172, 1.245e8, 18.8, 10**6)
M1000 = PopIIIStar(1000, 10**1.1090, 10**7.3047, 1.307e8, 10.49, 10**6)

g_per_GeV = 1.783 *10 ** (-24)

m_chi_sample = np.logspace(-6, 5, 100)
r = np.linspace(1, 2.7*10**11, 100)
m_chi_sample_cgs = []
for i in range(len(m_chi_sample)):
    m_chi_sample_cgs.append(g_per_GeV * m_chi_sample[i])
    



def omega_gould(z, mx, xi, sigma, star):
    #Central proton number density (cm^-3)
    nc = polytrope3_rhoc(star)*0.75/1.6726e-24
    
    a_plus = alpha('+', mx, vesc_r_poly(xi, star), z, xi, star)
    a_minus = alpha('-', mx, vesc_r_poly(xi, star), z, xi, star)
    
    b_plus = beta('+', mx, vesc_r_poly(xi, star), z, xi, star)
    b_minus = beta('-', mx, vesc_r_poly(xi, star), z, xi, star)
    
    #Pre factor for Omega in Gould using my notation
    omega_pre = (2*np.pi**(1/2))**-1 * (proton_speed(xi, star)/mu(mx))**2 * sigma*star.get_vesc_surf()/z * nc * eta_proton(xi)
    
    #Omega
    omega_val = omega_pre * (    mu(mx)*(a_plus*np.exp(-a_minus**2) - a_minus*np.exp(-a_plus**2)) + chi_func(a_minus, a_plus)*(mu(mx) - 2*mu(mx)*a_minus*a_plus - 2*mu_plus_minus('+', mx)*mu_plus_minus('-', mx)) + 2*mu_plus_minus('+', mx)**2 * chi_func(b_minus, b_plus)*np.exp( mu(mx)*(z**2 - vesc_r_poly(xi, star)**2)/proton_speed(xi, star)**2 )    )
    
    
    return omega_val


def nu_gould(mx, xi, star):
    return theta(xi)*mu(mx)/tau_fit(mx, star)

def xi_gould(mx, xi, star):
    return np.sqrt(mu_plus_minus('-', mx)**2 + nu_gould(mx, xi, star))

def rho_gould(mx, xi, star):
    return mu_plus_minus('+', mx)*mu_plus_minus('-', mx)/xi_gould(mx, xi, star)

def E_e_gould(mx, xi, star):
    return mx*1.783e-24*(vesc_r_poly(xi, star) * star.get_vesc_surf())**2 / 2

def E_c_gould(mx, xi, star):
    return E_e_gould(mx, xi, star)

def gamma_gould(pm, mx, xi, star):
    if (pm == '+'):
        ret_val = vesc_r_poly(xi, star)/proton_speed(xi, star)*(rho_gould(mx, xi, star) + xi_gould(mx, xi, star))
    else:
        ret_val = vesc_r_poly(xi, star)/proton_speed(xi, star)*(rho_gould(mx, xi, star) - xi_gould(mx, xi, star))
    return ret_val

def normalize_factor(mx, xi, star):
    ux = dm_speed(mx, star)
    ue_xi = vesc_r_poly(xi, star)    
    
    ret_val =  (sc.erf(ue_xi/ux) - 2/np.sqrt(np.pi)*ue_xi/ux*np.exp(-ue_xi**2/ux**2) )**-1 
    
    return ret_val

def R_gould(mx, xi, sigma, star):
    #Central proton number density (cm^-3)
    nc = polytrope3_rhoc(star)*0.75/1.6726e-24
    
    kb = 1.380649e-16 #Boltzmann constant in cgs Units (erg/K)
    
    T = (10**8) * theta(xi)
    Tx = tau_fit(mx, star) * 10**8
    mu_p = mu_plus_minus('+', mx)
    mu_m = mu_plus_minus('-', mx)
    nu = nu_gould(mx, xi, star)
    xi_g = xi_gould(mx, xi, star)
    Ee = E_e_gould(mx, xi, star)
    Ec = E_c_gould(mx, xi, star)
    g_p = gamma_gould('+', mx, xi, star)
    g_m = gamma_gould('-', mx, xi, star)
    a_p = alpha('+', mx, vesc_r_poly(xi, star), vesc_r_poly(xi, star), xi, star)
    a_m = alpha('-', mx, vesc_r_poly(xi, star), vesc_r_poly(xi, star), xi, star)
    b_p = beta('+', mx, vesc_r_poly(xi, star), vesc_r_poly(xi, star), xi, star)
    b_m = beta('-', mx, vesc_r_poly(xi, star), vesc_r_poly(xi, star), xi, star)
    
    R_pre = 2/np.pi * proton_speed(xi, star) * star.get_vesc_surf() * 1/np.sqrt(mu(mx)) * (T/Tx)**(3/2) * sigma * nc * eta_proton(xi)
    
    term1 = np.exp(-(mu_p/xi_g)**2 * (Ee/(kb*Tx))) * (mu(mx)*mu_m/(nu*xi_g)*(xi_g**2/nu - mu_p*mu_m/mu(mx)) + mu_p**3/(xi_g*(nu-mu(mx))))*chi_func(g_m, g_p) 
    
    term2 = np.exp(-Ec/(kb*Tx)) * mu(mx)/nu * ( a_p*a_m - 1/(2*mu(mx)) + mu_m**2*(1/mu(mx) - 1/nu) ) * chi_func(a_m, a_p)
    
    term3 = np.exp(-Ec/(kb*Tx)) * np.exp(-(Ee-Ec)/(kb*T)) * mu_p**2/(nu - mu(mx)) * chi_func(b_m, b_p)
    
    term4 = np.exp(-(Ec/(kb*Tx) + a_m**2)) * mu(mx)*a_p/(2*nu)
    
    term5 = np.exp(-(Ec/(kb*Tx) + a_p**2)) * mu(mx)*a_m/(2*nu)
    
    R_val = R_pre * (term1 + term2 - term3 - term4 + term5) * normalize_factor(mx, xi, star)
    
    return R_val



def R_gould_approx(mx, xi, sigma, star):

    #Central proton number density (cm^-3)
    n = polytrope3_rhoc(star)*0.75/1.6726e-24 * theta(xi)**3
    
    u = proton_speed(xi, star)
    
    R_val = 2*np.pi**(-1/2)*sigma*n*u*star.get_vesc_surf()*np.exp( -(vesc_r_poly(xi, star)/dm_speed(mx, star))**2 )

    
    return R_val


def upper_integrand_G(xi, mx, sigma, star):
    return xi**2 * nx_xi(mx, xi, star) * R_gould(mx, xi, sigma, star)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def lower_integrand_G(xi, mx, sigma, star):
    return xi**2 * nx_xi(mx, xi, star)

def evap_coeff_G(mx, sigma, star):
    xi1 = xis[-1]
    E = quad(upper_integrand_G, 0, xi1, args=(mx, sigma, star))[0]/quad(lower_integrand_G, 0, xi1, args=(mx, sigma, star))[0]
    
    return E

def upper_integrand_G_approx(xi, mx, sigma, star):
    return xi**2 * nx_xi(mx, xi, star) * R_gould_approx(mx, xi, sigma, star)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def lower_integrand_G_approx(xi, mx, sigma, star):
    return xi**2 * nx_xi(mx, xi, star)

def evap_coeff_G_approx(mx, sigma, star):
    xi1 = xis[-1]
    E = quad(upper_integrand_G_approx, 0, xi1, args=(mx, sigma, star))[0]/quad(lower_integrand_G_approx, 0, xi1, args=(mx, sigma, star))[0]
    
    return E


#Approximate DM Evaporation rate
def evap_coeff_Ilie_approx2(mx, sigma, star):
    #Central proton number density (cm^-3)
    nc = polytrope3_rhoc(star)*0.75/1.6726e-24
    
    #Edge of star in xis
    xi1 = xis[-1]

    #Central proton speed (Normalized to vesc)
    uc = proton_speed(xis[0], star)
    
    vesc = 1
    vesc_val = star.get_vesc_surf()

    #Dimensionless QTYs
    tau = tau_fit(mx, star)#Normalized DM Temperature
    mu = mx/0.93827 #Normalized DM mass
    
    #Analytic form of the DM evaporation rate
    E = 9/np.sqrt(np.pi) * 1/(xi1**3) * sigma * nc * uc *vesc_val * np.exp(-1/uc**2 * mu/tau * (1 + xi1/2)) * star.get_vol()/Vj_Eff_SP(star,mx , 1)
    
    return E


alpha_sample = np.zeros([len(r), len(m_chi_sample)])
beta_sample = np.zeros([len(r), len(m_chi_sample)])
gamma_sample = np.zeros([len(r), len(m_chi_sample)])
star = M100
for i in range(len(r)):
    for j in range(len(m_chi_sample)):
        #print("Calculating alpha, i = ", i, "/", len(r), ", j = ", j, "/", len(m_chi_sample_cgs))
        xi = 6.89*r[i]/r[-1]
        alpha_sample[i,j] = alpha('+', m_chi_sample[j], vesc_r_poly(xi, star), 0.5*vesc_r_poly(xi, star), xi, star)
        beta_sample[i,j] = beta('+', m_chi_sample[j], vesc_r_poly(xi, star), 0.5*vesc_r_poly(xi, star), xi, star)
        gamma_sample[i,j] = gamma_gould('+', m_chi_sample[j], xi, star)
            
# alpha
plt.pcolormesh(m_chi_sample, r, alpha_sample, cmap=palette1, shading='auto', edgecolors='face', norm=colors.LogNorm(vmin=abs(alpha_sample.min()), vmax=abs(alpha_sample.max())))
cbar = plt.colorbar()
plt.title("Gould Alpha Function: 10^2 GeV, 10^-43 cm^-2, 100 Msun")
plt.ylabel('$r$ [cm]')
plt.xlabel("$m_{\chi}$ [GeV]")
plt.xscale("log")
plt.savefig("caleb_alpha.pdf")
plt.clf()

# beta
plt.pcolormesh(m_chi_sample, r, beta_sample, cmap=palette1, shading='auto', edgecolors='face', norm=colors.LogNorm(vmin=abs(beta_sample.min()), vmax=abs(beta_sample.max())))
cbar = plt.colorbar()
plt.title("Gould Beta Function: 10^2 GeV, 10^-43 cm^-2, 100 Msun")
plt.ylabel('$r$ [cm]')
plt.xlabel("$m_{\chi}$ [GeV]")
plt.xscale("log")
plt.savefig("caleb_beta.pdf")
plt.clf()
            
# gamma
plt.pcolormesh(m_chi_sample, r, gamma_sample, cmap=palette1, shading='auto', edgecolors='face', norm=colors.LogNorm(vmin=abs(gamma_sample.min()), vmax=abs(gamma_sample.max())))
cbar = plt.colorbar()
plt.title("Gould Gamma Function: 10^2 GeV, 10^-43 cm^-2, 100 Msun")
plt.ylabel('$r$ [cm]')
plt.xlabel("$m_{\chi}$ [GeV]")
plt.xscale("log")
plt.savefig("caleb_gamma.pdf")
plt.clf()
    


<Figure size 864x576 with 0 Axes>